In [ ]:
import kagglehub

path = kagglehub.dataset_download("manith04/ddd-processed-1-training-frames-type-1")
#dddeeee  #ddewewewee  #manith04/gdgdfhfg
print(path)

/root/.cache/kagglehub/datasets/manith04/ddd-processed-1-training-frames-type-1/versions/1


In [ ]:
!kaggle datasets list -s "manith04"

ref                                     title                                size  lastUpdated                 downloadCount  voteCount  usabilityRating  
--------------------------------------  ----------------------------  -----------  --------------------------  -------------  ---------  ---------------  
manith04/slt-mobitel                    SLT_Mobitel                          4036  2025-05-22 11:24:53.007000              3          0  0.25             
manith04/abcdefg                        abcdefg                             94485  2025-05-23 08:48:36.640000              0          0  0.25             
manith04/genai-flux                     genAI_flux                         242974  2025-09-25 21:39:03.127000              0          0  0.25             
manith04/aigodmanith                    AIGODMANITH                         48456  2025-08-28 17:13:46.787000              0          0  0.25             
manith04/llm-finetune-manith            LLM_Finetune_Manith           

In [ ]:
!echo KAGGLE_API_TOKEN=KGAT_28e717b9ccd1d5f6908afdf6559d8cb4 | touch ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
%%bash
mkdir -p ~/.kaggle
echo '{"username":"manith04","key":"KGAT_28e717b9ccd1d5f6908afdf6559d8cb4"}' > ~/.kaggle/kaggle.json
chmod 600 ~/.kaggle/kaggle.json

#!kaggle datasets download -d manith04/manith-kitty-dataset -p ./data --unzip

In [ ]:
!kaggle datasets list

ref                                                             title                                                   size  lastUpdated                 downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  ------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
saidaminsaidaxmadov/chocolate-sales                             Chocolate Sales                                       468320  2026-01-04 14:23:35.490000          17213        307  1.0              
aliiihussain/amazon-sales-dataset                               Amazon_Sales_Dataset                                 1297759  2026-02-01 11:37:12.353000           3568         66  1.0              
ayeshasiddiqa123/student-perfirmance                            Student Academic Performance Dataset.                  96178  2026-01-06 12:08:32.540000           5749        108  1.0              
emirhanakk

In [ ]:
!uv python install 3.10
!export PATH="/root/.local/bin:$PATH"
!which python3.10



Python 3.10 is already installed
/usr/bin/python3.10


In [ ]:
%%bash
uv pip install --python python3.10 mediapipe==0.10.9 opencv-python tqdm matplotlib -q


In [ ]:
%%file ex.py
import os
os.environ["MPLBACKEND"] = "Agg"

import cv2
import mediapipe as mp
import numpy as np



import cv2
import mediapipe as mp
import numpy as np
from pathlib import Path

# Eye landmark indices (MediaPipe Face Mesh)
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

TARGET_SIZE = 224
MARGIN = 0.25  # expand eye box slightly

mp_face_mesh = mp.solutions.face_mesh


def crop_eye(image, landmarks, eye_indices):
    h, w, _ = image.shape

    pts = np.array([
        (int(landmarks[i].x * w), int(landmarks[i].y * h))
        for i in eye_indices
    ])

    x_min, y_min = pts.min(axis=0)
    x_max, y_max = pts.max(axis=0)

    # Expand bounding box
    bw = x_max - x_min
    bh = y_max - y_min
    cx = (x_min + x_max) // 2
    cy = (y_min + y_max) // 2

    size = int(max(bw, bh) * (1 + MARGIN))

    x1 = max(cx - size // 2, 0)
    y1 = max(cy - size // 2, 0)
    x2 = min(cx + size // 2, w)
    y2 = min(cy + size // 2, h)

    eye = image[y1:y2, x1:x2]

    if eye.size == 0:
        return None

    return cv2.resize(eye, (TARGET_SIZE, TARGET_SIZE))


# ----------- DEMO ON ONE IMAGE ----------------
image_path = "001_glasses_nonsleepyCombination_000000_0.jpg"
image = cv2.imread(image_path)

image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5
) as face_mesh:

    results = face_mesh.process(image_rgb)

    if not results.multi_face_landmarks:
        print("No face detected")
        exit()

    landmarks = results.multi_face_landmarks[0].landmark

    left_eye = crop_eye(image, landmarks, LEFT_EYE)
    right_eye = crop_eye(image, landmarks, RIGHT_EYE)

    if left_eye is not None:
        cv2.imwrite("001_glasses_nonsleepyCombination_000000_0_lefteye.jpg", left_eye)

    if right_eye is not None:
        cv2.imwrite("001_glasses_nonsleepyCombination_000000_0_righteye.jpg", right_eye)

print("Eye samples saved ✅")


Overwriting ex.py


In [ ]:
!python3.10 ex.py

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
Eye samples saved ✅


In [ ]:
%%file run.py

import os
os.environ.pop("MPLBACKEND", None)
os.environ["MPLBACKEND"] = "Agg"

import cv2
import mediapipe as mp
import numpy as np
from pathlib import Path
from tqdm import tqdm

# Eye landmark indices (FaceMesh)
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

TARGET_SIZE = 224
MARGIN = 0.3  # adjust if needed

mp_face_mesh = mp.solutions.face_mesh


def crop_eye(image, landmarks, eye_indices):
    h, w, _ = image.shape

    pts = np.array([
        [int(landmarks[i].x * w), int(landmarks[i].y * h)]
        for i in eye_indices
    ])

    x, y, bw, bh = cv2.boundingRect(pts)

    cx, cy = x + bw // 2, y + bh // 2
    size = int(max(bw, bh) * (1 + MARGIN))

    x1 = max(cx - size // 2, 0)
    y1 = max(cy - size // 2, 0)
    x2 = min(cx + size // 2, w)
    y2 = min(cy + size // 2, h)

    eye = image[y1:y2, x1:x2]

    if eye.size == 0:
        return None

    return cv2.resize(eye, (TARGET_SIZE, TARGET_SIZE))


def process_image(image_path, output_root, face_mesh):
    image = cv2.imread(str(image_path))
    if image is None:
        return

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image_rgb)

    if not results.multi_face_landmarks:
        return  # skip if no face

    landmarks = results.multi_face_landmarks[0].landmark

    left_eye = crop_eye(image, landmarks, LEFT_EYE)
    right_eye = crop_eye(image, landmarks, RIGHT_EYE)

    # Maintain folder structure
    rel_path = image_path.relative_to(INPUT_ROOT)
    save_folder = OUTPUT_ROOT / rel_path.parent
    save_folder.mkdir(parents=True, exist_ok=True)

    stem = image_path.stem

    if left_eye is not None:
        cv2.imwrite(str(save_folder / f"{stem}_lefteye.jpg"), left_eye)

    if right_eye is not None:
        cv2.imwrite(str(save_folder / f"{stem}_righteye.jpg"), right_eye)


def process_dataset(input_root, output_root):
    global INPUT_ROOT, OUTPUT_ROOT
    INPUT_ROOT = Path(input_root)
    OUTPUT_ROOT = Path(output_root)

    with mp_face_mesh.FaceMesh(
        static_image_mode=True,
        refine_landmarks=True,
        max_num_faces=1,
        min_detection_confidence=0.5
    ) as face_mesh:

        image_paths = list(INPUT_ROOT.rglob("*.jpg"))

        for img_path in tqdm(image_paths):
            process_image(img_path, OUTPUT_ROOT, face_mesh)


# ---------------- RUN ----------------

INPUT_IMAGES = "/root/.cache/kagglehub/datasets/manith04/ddd-processed-1-training-frames-type-1/versions/1"
OUTPUT_EYES = "/content/processed_training_1_eyes"

process_dataset(INPUT_IMAGES, OUTPUT_EYES)


Writing run.py


In [ ]:
!python3.10 run.py

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
100% 395691/395691 [1:21:03<00:00, 81.36it/s]


In [ ]:
!ls /root/.cache/kagglehub/datasets/manith04/ddd-processed-1-training-frames-type-1/versions/1

001  002  005  006  008  009  012  013	015  020
